### read_csv

file, sep , header, names , index_col, usecols, dtyes, parse_dates
- add duplicate suffix to columns
- header = None
- header = 0 , skip first row cols = [] , pass the colname 

### setting an index 
- parse_dates=True
- usecols , select the columns , saves processing time and memory ( add index also otherwise error)

### select rows 
nrows 
skiprows based on index position, nrows works after skiprows
- skiprows = lambda x:x%2 == 1

### missing values 
- na_values =[".","Missing"] --> NaN

### parsing dates parse_dates=["Date"],
infer_datetime_format=True

### dtype=({"Sales":"Int32"})

In [1]:
import pandas as pd 

In [11]:
columns = ["index", "Date", "Oil Price" , "Date2"]
string_slice = lambda x:int(x[-2:])

oil = pd.read_csv("./retail/oil.csv",
                  header = 0,
                # names = columns,
               #  parse_dates=["Date", "Date2"],
                # infer_datetime_format=True
                # index_col = "Date",
                  # dtype ={"index":"Int16"},
                 # converters = {"Date2":string_slice}
                 )
oil.rename(columns={"dcoilwtico": "Oil Price"}, inplace=True)
oil["index"] = oil.index

oil.head()

,date,Oil Price,index
0,2013-01-01,NaN,0
1,2013-01-02,93.14,1
2,2013-01-03,92.97,2
3,2013-01-04,93.12,3
4,2013-01-07,93.20,4


### converters 
currency = lambda x:f"${x}.00"
missing = lambda x:x if x not in ["MISSING","."] else 0

pd.read_csv(
"monthly_sales.csv",
converters={"Sales":missing,
"Sales.1":currency
}
)

### Exercise 1 streamlined transaction 